In [1]:
import datetime as dt
import meteomatics.api as api
def get_current_temperature(coordinates, specific_time):
    #meteomatics 用户名和密码
    username = 'none_lee_jine'
    password = 'JS1kb4k76V'
    parameters = ['t_2m:C']  # 只查询温度
    model = 'mix'
    df = api.query_time_series(
        coordinates,
        specific_time,
        specific_time,
        dt.timedelta(hours=1),
        parameters,
        username,
        password,
        model=model
    )
    return df

In [12]:
from datetime import datetime
import json
available_functions = {
    "get_current_temperature": get_current_temperature
}
def execute_function_call(function_name,arguments):
    function = available_functions.get(function_name,None)
    if function:
        # print(arguments)
        arguments=json.loads(arguments)
        # print(arguments)
         # 将字符串时间转换为 datetime 对象
        if 'specific_time' in arguments:
            dt_obj =  datetime.fromisoformat(arguments['specific_time'].replace('Z', '+00:00'))
            dt_str= dt_obj.strftime('%Y-%m-%d %H:%M:%S%z')
            dt_dt=dt.datetime.fromisoformat(dt_str)
            arguments['specific_time']=dt_dt
        # print(arguments)
        results = function(**arguments)
        print("sss",results)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_PROJECT_API_KEY"))


In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Fetch the current temperature of a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "coordinates": {    
                        "type": "array",
                         "items": {
                            "type": "array",
                            "items": {
                                "type": "number"
                            },
                            "minItems": 2,
                            "maxItems": 2
                        },
                        "description": "Array of [latitude, longitude] pairs, e.g., [[33, 44]]."
                    },
                    "specific_time": {
                        "type": "string",
                        "format": "date-time",
                        "description": "The specific time to get the temperature.e.g., '2024-12-25 01:00:00+00:00'"
                    }
                },
                "required": ["coordinates","specific_time"]
            }
        }
    }
]

In [18]:
assistant = client.beta.assistants.create(
  name="weather bot",
  instructions="You are an weather bot. Use the provided functions to answer questions. Synthesise answer based on provided function output and be consise",
  model="gpt-4",
  tools = tools
)

In [19]:
#创建thread,messages,run
def create_message_and_run(assistant,query,thread=None):
  if not thread:
    thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=query
  )
  run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
  )
  return run,thread

In [20]:
#获取required_action状态下所需的函数名，参数，函数id
def get_function_details(run):

  print("\nrun.required_action\n",run.required_action)

  function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
  arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
  function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

  print(f"function_name: {function_name} and arguments: {arguments}")

  return function_name, arguments, function_id


#把函数执行结果提交到run
def submit_tool_outputs(run,thread,function_id,function_response):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
      {
        "tool_call_id": function_id,
        "output": str(function_response),
      }
    ]
    )
    return run

In [21]:
query = "I want to know the current temperature in Beijing.Today is 2024-12-25"
run,thread = create_message_and_run(assistant=assistant,query=query)



In [22]:
run

Run(id='run_jJIJx7l6n1VU5fB2KDsPtXEm', assistant_id='asst_fVYmhSszPXsDRFLvpLqnEnYF', cancelled_at=None, completed_at=None, created_at=1735106661, expires_at=1735107261, failed_at=None, incomplete_details=None, instructions='You are an weather bot. Use the provided functions to answer questions. Synthesise answer based on provided function output and be consise', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_CRydEx4mjsSf7lQc89d4DZJu', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_current_temperature', description='Fetch the current temperature of a given location.', parameters={'type': 'object', 'properties': {'coordinates': {'type': 'array', 'items': {'type': 'array', 'items': {'type': 'number'}, 'minItems': 2, 'maxItems': 2}, 'description': 'Array of [latitude,

In [23]:
import time
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print("run status", run.status)

    if run.status=="requires_action":

        function_name, arguments, function_id  = get_function_details(run)

        function_response = execute_function_call(function_name,arguments)

        run = submit_tool_outputs(run,thread,function_id,function_response)

        continue
    if run.status=="completed":

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)

        user_input = input()
        if user_input == "STOP":
          break

        run,thread = create_message_and_run(assistant=assistant,query=user_input,thread=thread)

        continue;
    time.sleep(1)

run status requires_action

run.required_action
 RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_dIXapHQGBqZfg4TaxUTkeppf', function=Function(arguments='{\n  "coordinates": [[39.9042, 116.4074]],\n  "specific_time": "2024-12-25 00:00:00+00:00"\n}', name='get_current_temperature'), type='function')]), type='submit_tool_outputs')
function_name: get_current_temperature and arguments: {
  "coordinates": [[39.9042, 116.4074]],
  "specific_time": "2024-12-25 00:00:00+00:00"
}
sss                                             t_2m:C
lat     lon      validdate                        
39.9042 116.4074 2024-12-25 00:00:00+00:00    -7.3
run status in_progress
run status completed
The current temperature in Beijing on 2024-12-25 is -7.3°C.
